<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/cs_small_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-u_ioobv6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-u_ioobv6
  Resolved https://github.com/huggingface/transformers to commit 35551f9a0f66a22de4971b4a51b3c172d3b87f95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8049176 sha256=43c802c3e1c2634ebb9fbf02323dfd90681fff88a88743c49415b3edba6c94c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ywy6w9dl/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm


In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "cs", split="validation")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1103
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=fe8baf0da55fcc40249234e428b8954ab5fb54d12cd7b7f2a33c0bacd85aaed6
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [7]:
model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 133MiB/s]


In [9]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [10]:
text

,0
0,je to opravdu tak závažný fakt že k tomu evrop...
1,v malých státech to naopak povede k tomu že by...
2,hlasovala jsem proti závaznému charakteru náro...
3,zajímavá je angažovanost této země v mezinárod...
4,tento výzkum je důležitý i pro rozvoj nových m...
...,...
1098,průběh dalšího projednávání v trialogu tak vid...
1099,dále je zřejmé že vzhledem k množství pozměňov...
1100,jižního koridoru.
1101,proto tak záleží na připravenosti absolventů n...


In [11]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [12]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,je to opravdu tak závažný fakt že k tomu evrop...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00103759765625, -0.0047607421875, -0.00054...",16000
1,male,v malých státech to naopak povede k tomu že by...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.01129150390625, -0.04022216796875, -0.07479...",16000
2,female,hlasovala jsem proti závaznému charakteru náro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.011383056640625, -0.0126953125, -0.03192138...",16000
3,male,zajímavá je angažovanost této země v mezinárod...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002227783203125, 0.0008544921875, 0.000732...",16000
4,male,tento výzkum je důležitý i pro rozvoj nových m...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004669189453125, 0.003143310546875, 0.00305...",16000
...,...,...,...,...,...
1098,male,průběh dalšího projednávání v trialogu tak vid...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000274658203125, 0.0001220703125, -3.051757...",16000
1099,female,dále je zřejmé že vzhledem k množství pozměňov...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002105712890625, -0.002105712890625, -0.00...",16000
1100,male,jižního koridoru.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00177001953125, 0.00067138671875, 0.0037536...",16000
1101,female,proto tak záleží na připravenosti absolventů n...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, 0.000335693359375, -0.00015258...",16000


In [13]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,je to opravdu tak závažný fakt že k tomu evrop...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00103759765625, -0.0047607421875, -0.00054...",16000
1,male,v malých státech to naopak povede k tomu že by...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.01129150390625, -0.04022216796875, -0.07479...",16000
3,male,zajímavá je angažovanost této země v mezinárod...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002227783203125, 0.0008544921875, 0.000732...",16000
4,male,tento výzkum je důležitý i pro rozvoj nových m...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004669189453125, 0.003143310546875, 0.00305...",16000
5,male,to je trochu rozdíl oproti tomu co zde bylo pa...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.364776611328125, -0.3585205078125, -0.3042...",16000
...,...,...,...,...,...
1091,male,pane předsedající nejsem myslivec ani sportovn...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, -0.0001220703125, 0.00070...",16000
1093,male,dopředu vám říkám že se tu nebudu pouštět do f...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000762939453125, 0.001312255859375, -0.0002...",16000
1098,male,průběh dalšího projednávání v trialogu tak vid...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000274658203125, 0.0001220703125, -3.051757...",16000
1100,male,jižního koridoru.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00177001953125, 0.00067138671875, 0.0037536...",16000


In [14]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
2,female,hlasovala jsem proti závaznému charakteru náro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.011383056640625, -0.0126953125, -0.03192138...",16000
12,female,obecně samozřejmě podporuji flexibilní přístup...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.012725830078125, 0.002166748046875, 0.0122...",16000
13,female,vyjednaná podoba je vyváženým kompromisem i pr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02734375, 0.01495361328125, 0.042724609375...",16000
16,female,problém vidím také v tom že se naše firmy nemo...,/root/.cache/huggingface/datasets/downloads/ex...,"[9.1552734375e-05, -0.0009765625, -0.000457763...",16000
17,female,proto bych moc ráda apelovala na členské státy...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -9.1552734375e-05, -0.0002...",16000
...,...,...,...,...,...
1095,female,a my chceme aby orgány činné v trestním řízení...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.018951416015625, 0.01361083984375, -0.01022...",16000
1096,female,pane předsedající pane komisaři pane předsedo ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001953125, -0.000457763671875, 0.000793457...",16000
1097,female,že hlavním zdrojem chyb pak bohužel byly nespr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.0003662109375, 0.0006408691...",16000
1099,female,dále je zřejmé že vzhledem k množství pozměňov...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002105712890625, -0.002105712890625, -0.00...",16000


In [15]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="cs")
    return result["text"]

In [16]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [20]:
total_wer = 0
num_sample = 0


for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3333333333333333
Processed 2 files, WER: 0.3333333333333333
Processed 3 files, WER: 0.26666666666666666
Processed 4 files, WER: 0.36
Processed 5 files, WER: 0.46153846153846156
Processed 6 files, WER: 0.35714285714285715
Processed 7 files, WER: 0.4166666666666667
Processed 8 files, WER: 0.5384615384615384
Processed 9 files, WER: 0.30434782608695654
Processed 10 files, WER: 0.9230769230769231
Processed 11 files, WER: 0.29411764705882354
Processed 12 files, WER: 0.3023255813953488
Processed 13 files, WER: 0.4117647058823529
Processed 14 files, WER: 0.2
Processed 15 files, WER: 0.375
Processed 16 files, WER: 0.3103448275862069
Processed 17 files, WER: 0.3333333333333333
Processed 18 files, WER: 0.07142857142857142
Processed 19 files, WER: 0.2
Processed 20 files, WER: 0.41379310344827586
Processed 21 files, WER: 0.4117647058823529
Processed 22 files, WER: 0.5
Processed 23 files, WER: 0.2692307692307692
Processed 24 files, WER: 0.23333333333333334
Processed 25 file

In [21]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3939393939393939
Processed 2 files, WER: 0.6666666666666666
Processed 3 files, WER: 0.6
Processed 4 files, WER: 0.375
Processed 5 files, WER: 0.5
Processed 6 files, WER: 0.3333333333333333
Processed 7 files, WER: 0.4444444444444444
Processed 8 files, WER: 0.4666666666666667
Processed 9 files, WER: 0.35294117647058826
Processed 10 files, WER: 0.3333333333333333
Processed 11 files, WER: 0.3
Processed 12 files, WER: 0.2
Processed 13 files, WER: 0.42424242424242425
Processed 14 files, WER: 0.42857142857142855
Processed 15 files, WER: 0.391304347826087
Processed 16 files, WER: 0.7
Processed 17 files, WER: 0.3
Processed 18 files, WER: 1.0
Processed 19 files, WER: 0.3157894736842105
Processed 20 files, WER: 0.6923076923076923
Processed 21 files, WER: 0.27586206896551724
Processed 22 files, WER: 0.35714285714285715
Processed 23 files, WER: 0.0
Processed 24 files, WER: 0.5185185185185185
Processed 25 files, WER: 0.4
Processed 26 files, WER: 0.25
Processed 27 files, WER